## Flopy data types, head and budget files

In order to access the binary data outputs created by MODFLOW, flopy uses [utility modules](https://modflowpy.github.io/flopydoc/binaryfile.html).

The class *flopy.utils.binaryfile.HeadFile* creates a Headfile object with mutliple functions to view and retrieve the data. To confuse us more, the flopy resource describes it as "a record array consisting of headers, which are record arrays of the modflow header information (kstp, kper, pertim, totim, text, nrow, ncol, ilay)"

### HeadFile - for steady state models
In steady state there is one time so there is only one set of head values so *get_alldata()* function is the easisest way to retrieve them, but using *get_data()* can reduce the dimensions of the array.

In [1]:
import flopy
import numpy as np

headobj = flopy.utils.binaryfile.HeadFile('Box_Model.hds') #Get head values for simple steady state model
#headobj.list_records() #shows records that are available
h = headobj.get_alldata() #Creates an array (ntimes, nlay, nrow, ncol)
h2 = headobj.get_data(totim=1, mflay=0) #Creates array (nrow, ncol)
print (h[0,0,0,:]) #print heads for 1st row, all columns
print (h2[0,:]) #print heads for 1st row, all columns

flopy is installed in c:\users\yoko\appdata\local\programs\python\python38-32\lib\site-packages\flopy
[7.        7.1414423 7.280136  7.4162354 7.5498815 7.6812015 7.810313
 7.9373245 8.062335  8.185435  8.306711  8.426242  8.5441    8.660355
 8.775068  8.888301  9.00011   9.110545  9.219659  9.327496  9.4341
 9.539513  9.643773  9.746919  9.848984 ]
[7.        7.1414423 7.280136  7.4162354 7.5498815 7.6812015 7.810313
 7.9373245 8.062335  8.185435  8.306711  8.426242  8.5441    8.660355
 8.775068  8.888301  9.00011   9.110545  9.219659  9.327496  9.4341
 9.539513  9.643773  9.746919  9.848984 ]


### Head file for different times
Instead of *get_alldata()*, we can use *get_data()* and specify the simulation time or timestep and period (list) to get the heads for that specific parameter.

In [2]:
# read in the heads
headobj = flopy.utils.binaryfile.HeadFile('ET_Model_CheckET2.hds')

#headobj.list_records()

#get data for specified simulation time
h = headobj.get_data(totim=100) #get data for specified simulation time
#print(h)

#NOTE: Can be helpful to get all the times in the head file using get_times()
t = headobj.get_times()

#get data for a specific timestep and stressperiod(kstp, kper)
kstpkper = headobj.get_kstpkper()  

#print(kstpkper)

#Find the heads across the domain for a given timestep and stress period
h = headobj.get_data(kstpkper = (1,1))
#print(h)

## Head File - time series
Use the *get_ts()* function to retrieve head values for a cell - input for single cell as a tuple of (nlay, nrow, ncol), or list of tuples (for multiple cells)

In [3]:
#Timeseries
cell = (0,0,0) #single cell nlay, nrow, ncol 
ts = headobj.get_ts(cell) #returns an array (timestep, head value)
#print(ts)

cells = [(0,0,0), (0,1,1)] #list of tuples for two cells
ts = headobj.get_ts(cells) #returns an array (timestep, head value for cell 1, head value for cell 2)

#print(ts)

### Sticking to your Budget
Flopy has the class *flopy.utils.binaryfile.CellBudgetFile()* to create objects with data for to calculate the water budget at each cell.This imports the data from the ".cbc" file.

In [4]:
#create the budjet object from the data in the cbc file
budgobj = flopy.utils.binaryfile.CellBudgetFile('ET_Model_CheckET.cbc')

recnames = budgobj.get_unique_record_names() #Find all the unique records that occur in the model output
print(recnames)

et = budgobj.get_data(text='ET', totim=1.0) #returns an array of records with text identifier of ET in format (idx, nlay, nrow, ncol)
recharge = budgobj.get_data(text='RECHARGE', totim=1.0) #returns an array of flow rates for front face of cells 
print(et[0][1][30][49]) #print ET FLux at cell from idx = 0, nlay = 1, nrow = 1, ncol = 49
print(recharge[0][1][0][49]) #print recharge Flux for same cell

#Sum all ET and Recharge Cells (from HW4)
print(np.sum(et[0][1]))
print(np.sum(recharge[0][1]))

[b'   CONSTANT HEAD', b'FLOW RIGHT FACE ', b'FLOW FRONT FACE ', b'              ET', b'        RECHARGE']
-0.74454135
1.0
-1789.3469
2450.0


## Budget File - multiple times
Just like the headfile, we can pull heads for different times (or timestep and stressperiod List) or a time series for a cell. In addition to the time or cell location we need to identify the record name (ET, Recharge, etc)

In [5]:
#Find the flux across the domain for a given timestep and stress period
et = budgobj.get_data(kstpkper = (0,0), text='ET')
print(et[0][1])


#Timeseries
cell = (0,0,0) #single cell nlay, nrow, ncol 
ts = budgobj.get_ts(idx=cell, text='ET') #returns an array (timestep, ETflux)
print(ts)

cells = [(0,0,0), (0,1,1)] #list of tuples for two cells
ts = budgobj.get_ts(idx=cells, text='ET') #returns an array (timestep, ET flux for cell 1, ET flux for cell 2)

print(ts)

[[ 0.         -0.70188504 -0.70372266 ... -0.7444384  -0.7445071
  -0.74454135]
 [ 0.         -0.70188504 -0.70372266 ... -0.7444384  -0.7445071
  -0.74454135]
 [ 0.         -0.70188504 -0.70372266 ... -0.7444384  -0.7445071
  -0.74454135]
 ...
 [ 0.         -0.70188504 -0.70372266 ... -0.7444384  -0.7445071
  -0.74454135]
 [ 0.         -0.70188504 -0.70372266 ... -0.7444384  -0.7445071
  -0.74454135]
 [ 0.         -0.70188504 -0.70372266 ... -0.7444384  -0.7445071
  -0.74454135]]
[[1. 0.]]
[[ 1.          0.         -0.70188504]]


### Ploting your information
One flopy function worth mentioning is the *flopy.plot.PlotMapView()* This is Flopy's way of reading outputs and graphically representing the file types above.